In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import re
import joblib
import string

In [2]:
fake_news = pd.read_csv('..\\Datasets\\Fake.csv')
fake_news.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [3]:
true_news = pd.read_csv('..\\Datasets\\True_news.csv')
true_news.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [4]:
fake_news['label'] = 0
true_news['label'] = 1

In [5]:
data = pd.concat([fake_news, true_news], axis=0)

In [6]:
data.sample(5)

,title,text,subject,date,label
8031,"For some Democrats, it's voting for Clinton - ...",NEW YORK (Reuters) - One would expect voters f...,politicsNews,"September 25, 2016",1
9971,Treasury Secretary’s Actress Wife Answers Crit...,Treasure Secretary Steve Mnuchin s wife is ans...,politics,"Sep 5, 2017",0
16392,CAN WE ADD DIRTY MONEY ‘LAUNDERING’ To The Oba...,A member of the House Intelligence Committee i...,Government News,"Sep 19, 2016",0
14329,Head of Syrian opposition's HNC resigns: state...,BEIRUT (Reuters) - The head of the Syrian oppo...,worldnews,"November 20, 2017",1
3075,Leading U.S. House Republican rejects tax cuts...,WASHINGTON (Reuters) - The top Republican on t...,politicsNews,"June 22, 2017",1


In [7]:
data =data.drop(['title','subject','date'], axis=1)


In [8]:
data.reset_index(inplace=True)

In [9]:
data.sample(5)

,index,text,label
32297,8816,WASHINGTON (Reuters) - Republican Party chief ...,1
18002,18002,"Kris Kobach For years, the mainstream media...",0
9312,9312,Former Baywatch star Pamela Anderson tried t...,0
18926,18926,Assimilation fail A French Muslim has been arr...,0
2507,2507,"How can the Israelis, or any Jewish person for...",0


In [10]:
def clean_text(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W", " ", text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [11]:
data['text'] = data['text'].apply(clean_text)

In [12]:
data.isnull().sum()

index    0
text     0
label    0
dtype: int64

In [13]:
# Remove rows with empty text
data = data[data['text'].str.strip() != '']
data.reset_index(drop=True, inplace=True)

In [14]:
data['text'] = data['text'].astype(str)

In [15]:
x=data['text']
y=data['label']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

In [16]:
vectorizer = TfidfVectorizer()
x_train_vectors = vectorizer.fit_transform(x_train)
x_test_vectors = vectorizer.transform(x_test)

In [17]:
lr = LogisticRegression()
lr.fit(x_train_vectors, y_train)

LogisticRegression()

In [18]:
predictions = lr.predict(x_test_vectors)
lr.score(x_test_vectors, y_test)

0.9846390168970814

In [19]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      5727
           1       0.98      0.99      0.98      5340

    accuracy                           0.98     11067
   macro avg       0.98      0.98      0.98     11067
weighted avg       0.98      0.98      0.98     11067



In [20]:
joblib.dump(vectorizer, 'vectorizer.jb')
joblib.dump(lr, 'model.jb')

['model.jb']